# Script-Languages-Container UI

<b>This notebook is not supposed to be used on its own.<b>


In [2]:
from exasol.nb_connector.utils import upward_file_search

# This NB may be running from various locations in the NB hierarchy.
# Need to search for the styles NB from the current directory upwards.

%run {upward_file_search('utils/ui_styles.ipynb')}
%run {upward_file_search('utils/popup_message_ui.ipynb')}

Output()

In [3]:
from exasol.nb_connector.ai_lab_config import AILabConfig

slc_source_slc_repo_option_value = "Clone script languages release repository"

from ipyfilechooser import FileChooser
from pathlib import Path
from exasol.nb_connector.secret_store import Secrets


NOTEBOOKS_DIRECTORY = str(Path().parent.parent)

def clone_slc_repo(conf: Secrets) -> bool:
    return conf.get(AILabConfig.slc_source) == slc_source_slc_repo_option_value

def get_slc_source_selection_ui(conf: Secrets) -> widgets.Widget:
    """
    Creates a UI form for choosing the flavor source
    """

    ui_look = get_config_styles()
    ui_look.input_layout.max_width = "500px"
    
    slc_sources = [slc_source_slc_repo_option_value, "Use the existing clone"]
    default_value = conf.get(AILabConfig.slc_source, slc_source_slc_repo_option_value)

    flavor_source_selector = widgets.RadioButtons(options=slc_sources, value=default_value, 
                                                  layout=ui_look.input_layout, style=ui_look.input_style)
    select_btn = widgets.Button(description='Select', style=ui_look.button_style, layout=ui_look.button_layout)
    header_lbl = widgets.Label(value='Flavor choice', style=ui_look.header_style, layout=ui_look.header_layout)


    def select_flavor_source(btn):
        conf.save(AILabConfig.slc_source, flavor_source_selector.value)
        btn.icon = 'check'

    def on_value_change(change):
        select_btn.icon = 'pen'

    select_btn.on_click(select_flavor_source)
    flavor_source_selector.observe(on_value_change, names=['value'])

    group_items = [header_lbl, widgets.Box([flavor_source_selector], layout=ui_look.row_layout)]
    items = [widgets.Box(group_items, layout=ui_look.group_layout), select_btn]
    ui = widgets.Box(items, layout=ui_look.outer_layout)
    return ui

def get_slc_target_dir_ui(conf: Secrets) -> widgets.Widget:
    """
    Creates a UI form for editing the Script-Languages-Container repository configuration.
    """
    default_target_dir = NOTEBOOKS_DIRECTORY
    target_dir_chooser_widget = FileChooser(path=default_target_dir, select_default=True)
    target_dir_chooser_widget.show_only_dirs = True
    target_dir_chooser_widget.sandbox_path = NOTEBOOKS_DIRECTORY
    
    inputs = [
        [
            ('Target Base Directory', target_dir_chooser_widget, AILabConfig.slc_target_dir),
        ]
    ]

    ui_look = get_config_styles()
    ui_look.row_layout.max_width = "500px"
    ui_look.group_layout.max_width = "500px"
    save_btn = widgets.Button(description='Save', style=ui_look.button_style, layout=ui_look.button_layout)
    header_lbl = widgets.Label(value='Target Directory', style=ui_look.header_style, layout=ui_look.header_layout)

    def save_configuration(btn):
        target_dir = Path(target_dir_chooser_widget.selected) / "script_languages_release"
        conf.save(AILabConfig.slc_target_dir, str(target_dir))
        btn.icon = 'check'

    def on_value_change(change):
        save_btn.icon = 'pen'

    save_btn.on_click(save_configuration)

    # Apply the styles and layouts to the input fields
    target_dir_chooser_widget.observe(on_value_change, names=['value'])

    group_items = [header_lbl, widgets.Box([target_dir_chooser_widget], layout=ui_look.row_layout)]
    items = [widgets.Box(group_items, layout=ui_look.group_layout), save_btn]
    ui = widgets.Box(items, layout=ui_look.outer_layout)
    

    if clone_slc_repo(conf):
        return ui

def get_existing_slc_ui(conf: Secrets) -> widgets.Widget:
    """
    Creates a UI form for choosing the existing script-languages repository.
    """
    if clone_slc_repo(conf):
        return
    default_src_dir = conf.get(AILabConfig.slc_target_dir, '')
    select_default = True if default_src_dir else False
    src_dir_chooser_widget = FileChooser(path=default_src_dir, select_default=select_default)
    src_dir_chooser_widget.show_only_dirs = True
    src_dir_chooser_widget.sandbox_path = NOTEBOOKS_DIRECTORY
    
    inputs = [
        [
            ('Flavor source directory', src_dir_chooser_widget, AILabConfig.slc_target_dir),
        ]
    ]

    ui_look = get_config_styles()
    ui_look.row_layout.max_width = "500px"
    ui_look.group_layout.max_width = "500px"
    save_btn = widgets.Button(description='Save', style=ui_look.button_style, layout=ui_look.button_layout)
    header_lbl = widgets.Label(value='Existing script-languages directory', style=ui_look.header_style, layout=ui_look.header_layout)


    def save_configuration(btn):
        target_dir = Path(src_dir_chooser_widget.selected)
        if not (target_dir / "flavors").is_dir():
            popup_message("Invalid directory. You need to choose a valid script-languages repository.")
        conf.save(AILabConfig.slc_target_dir, str(src_dir_chooser_widget.selected))
        btn.icon = 'check'

    def on_value_change(change):
        save_btn.icon = 'pen'

    save_btn.on_click(save_configuration)

    # Apply the styles and layouts to the input fields
    src_dir_chooser_widget.observe(on_value_change, names=['value'])

    group_items = [header_lbl, widgets.Box([src_dir_chooser_widget], layout=ui_look.row_layout)]
    items = [widgets.Box(group_items, layout=ui_look.group_layout), save_btn]
    ui = widgets.Box(items, layout=ui_look.outer_layout)
    

    if not clone_slc_repo(conf):
        return ui
